In [1]:
import numpy as np
import pandas as pd
import time
import re

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import NoSuchElementException

### Extract All The Links

In [2]:
PATH = "C:\Program Files (x86)\web_driver\chromedriver.exe"
driver = webdriver.Chrome(PATH)

# start at this page
driver.get('https://dialnet.unirioja.es/tesis/fechalectura?inicio=2020')

In [4]:
%%time
absLinks=[]
max_pages = 0
while True:
    link1 = driver.find_elements_by_css_selector("span.titulo")
    for tags in link1:
        link2 = tags.find_elements_by_tag_name("a")
        for link3 in link2:
            absLinks.append(link3.get_attribute("href"))
    
    # extract links
    try:
        time.sleep(7)
        driver.find_element_by_xpath('//*[@id="pieDeListadoDeBusquedaDeAutores"]/ul/li[4]/a').click() 
        max_pages += 1
        if max_pages == 200:
            break
    # stop if no more pages available
    except NoSuchElementException:
        break

Wall time: 30min 2s


### Get Contents From Each Link

In [5]:
data = []
for link in absLinks:
    time.sleep(5)
    driver.get(link)

    try:
        contents = driver.find_elements_by_xpath('//*[@id="resumen"]')
        for content in contents:
            language = content.text

    except NoSuchElementException:
        break

    #collect the data
    lang= {'content': language}
    data.append(lang)

KeyboardInterrupt: 

In [40]:
en_sp = pd.DataFrame(data)

In [41]:
len(en_sp)

178

### Data Cleaning

In [42]:
# remove \n 
en_sp['content'] = en_sp['content'].apply(lambda x: re.sub(r'\n', '', x))

# return only rows with 'Espanol' in the text (because it contains english translate too)
en_sp['Spanish'] = en_sp['content'].apply(lambda x: x if 'Español' in x else 0)

# remove rows with 0
en_sp = en_sp[en_sp['Spanish'] != 0]

# create two new variables: Spanish_text, English_text to separate the translations
en_sp['Spanish_text'] = en_sp['Spanish'].apply(lambda x: x.split('English')[0])
en_sp['English_text'] = en_sp['Spanish'].apply(lambda x: x.split('English')[-1])

# remove 'Galego' translation in the English_text variable
en_sp['English_text'] = en_sp['English_text'].apply(lambda x: x.split('Galego')[0] if 'Galego' in x else x)

en_sp['Spanish_text'] = en_sp['Spanish_text'].apply(lambda x: x.split('Español')[-1])

# remove any text that contains 'Español' in the English_text
en_sp['English_text'] = en_sp['English_text'].apply(lambda x: 0 if 'Español' in x else x)

# remove rows with 0
en_sp = en_sp[en_sp['English_text'] != 0]


In [43]:
# make sure Spanish_text and English_text do not have the same value
en_sp = en_sp[en_sp['Spanish_text'] != en_sp['English_text']]
len(en_sp)

37

In [44]:
# format dataframe
en_sp.reset_index(inplace=True)
en_sp.drop(['index', 'content', 'Spanish'], axis=1, inplace=True)

In [45]:
en_sp

,Spanish_text,English_text
0,ANTECEDENTES: La depresión se considera un pro...,BACKGROUND: Depression is considered a global ...
1,La crisis global que se inició en el 2008 es l...,The global crisis that began in 2008 is the wo...
2,ANTECEDENTES: La depresión se considera un pro...,BACKGROUND: Depression is considered a global ...
3,La crisis global que se inició en el 2008 es l...,The global crisis that began in 2008 is the wo...
4,Muchas y muy diferentes son las propuestas que...,A huge number of proposals have been developed...
5,En el siguiente artículo se pretende exponer u...,The aim of this article is to provide a panora...
6,La inspiración de la presente tesis gira entor...,The debate on the emissions control instrument...
7,Los lesionados medulares con alguna capacidad ...,Spinal-cord-injured (SCI) subjects with some l...
8,"En España, la teoría de la evolución se aborda...","In Spain, the theory of evolution is addressed..."
9,La transición de la mesoescala a la submesoesc...,The transition from mesoscale to submesoscale ...


In [12]:
en_sp['Spanish_text'][30]

'El reemplazo articular es un procedimiento que mejora la vida de millones de personas al año, proporcionando alivio del dolor, restableciendo la función articular y aportando independencia. En los próximos veinte años se espera que el número de artroplastias aumente en más de un 150%, lo que traerá consigo un aumento de las complicaciones asociadas, como el aflojamiento, lesiones vasculares, inestabilidad, mala cicatrización de la herida quirúrgica, o infecciones.Las infecciones de prótesis articulares, aunque infrecuentes, suponen una condición devastadora para el paciente y un alto coste para el sistema sanitario. La mayoría están causadas por bacterias y un 1-2% están causadas por hongos, entre los que destacan algunas especies del género Candida. Actualmente, existe gran cantidad de literatura relacionada con las primeras, con guías terapéuticas disponibles para el manejo y tratamiento de los pacientes. En cambio, debido a la baja frecuencia, las distintas características clínicas

In [13]:
en_sp['English_text'][30]

'Joint replacement is a procedure that improves the life of millions of people every year, providing pain relief, independence and restoring joint function. It is expected that the number of arthroplasties will increase by more than 150% in the next twenty years, which consequently will lead to an increase in the associated complications, such as loosening, vascular injuries, instability, poor surgical wound healing or infections.Prosthetic joint infections, despite being infrequent, represent a devastating condition for the patient and entail elevated costs for the healthcare system. The majority of these infections are caused by bacteria and 1-2% are caused by fungi, being Candida species the most frequently isolated.There currently exist large amounts of literature describing bacterial prosthetic joint infections and therapeutic guidelines are available for patient management and treatment. In contrast, standardized therapeutic guidelines are not available for fungal prosthetic join

In [46]:
en_sp.to_csv('lang_doc2.csv', index=False)